# Installing Packages in all Systems at once

In [ ]:
import subprocess

# Remote system configs
remote_configs = {
    "cadencea19@172.16.121.14": {"password": "caduser@123", "enabled": True},
    "cadencea18@172.16.121.11": {"password": "caduser@123", "enabled": True},
    "cadencea22@172.16.121.25": {"password": "caduser@123", "enabled": True},
    "cadencea24@172.16.121.30": {"password": "caduser@123", "enabled": True},

    "cadencea2@172.16.121.8": {"password": "caduser@123", "enabled": True},
    "cadencea6@172.16.121.66": {"password": "caduser@123", "enabled": True},
    "cadencea16@172.16.121.75": {"password": "caduser@123", "enabled": True},
}

# List of required packages
packages = ["pandas", "numpy", "Pillow", "tqdm"]

# Remote command to install packages without root
install_command = f"pip3 install --user {' '.join(packages)}"

# Also export PATH for ~/.local/bin so scripts work properly
env_fix = 'export PATH="$HOME/.local/bin:$PATH"'

# Final command to execute remotely
full_command = f"{env_fix} && {install_command}"

# Run the command on all enabled systems
for user_host, config in remote_configs.items():
    if not config.get("enabled", False):
        continue

    password = config["password"]
    print(f"\n📡 Installing packages on {user_host}...")

    ssh_cmd = [
        "sshpass", "-p", password,
        "ssh", "-o", "StrictHostKeyChecking=no",
        user_host,
        full_command
    ]

    try:
        subprocess.run(ssh_cmd, check=True)
        print(f"✅ Successfully installed packages on {user_host}")
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install on {user_host}: {e}")


In [3]:
import subprocess

remote_configs = {
    "cadencea12@172.16.121.82": {"password": "caduser@123", "subdir": ["array_config_1", "array_config_2"]},
    "cadencea1@172.16.121.72": {"password": "caduser@123", "subdir": ["array_config_5", "array_config_6"]},
    "cadencea8@172.16.121.78": {"password": "caduser@123", "subdir": ["array_config_7", "array_config_8"]},
    "cadencea10@172.16.121.80": {"password": "caduser@123", "subdir": ["array_config_9", "array_config_10"]},
    "cadencea15@172.16.121.6": {"password": "caduser@123", "subdir": ["array_config_11", "array_config_12"]},
    "cadencea14@172.16.121.84": {"password": "caduser@123", "subdir": ["array_config_13", "array_config_14"]},
    "imt2022556_nishit@172.16.121.37": {"password": "$@Rl@1234", "subdir": ["array_config_3", "array_config_4"]},
}

def rename_remote_dirs(user_host, password, new_dirs):
    user, host = user_host.split("@")
    base_path = f"/home/{user}/Documents/strassen"
    
    cmd = f"""
        cd {base_path} && \
        mv array_4096_1 {new_dirs[0]} && \
        mv array_4096_2 {new_dirs[1]}
    """
    
    ssh_command = [
        "sshpass", "-p", password,
        "ssh", "-o", "StrictHostKeyChecking=no",
        "-o", "UserKnownHostsFile=/dev/null",
        f"{user_host}",
        cmd
    ]

    try:
        print(f"🔄 Renaming folders in {user_host}...")
        subprocess.run(ssh_command, check=True)
        print(f"✅ Renamed to: {new_dirs[0]}, {new_dirs[1]}")
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed on {user_host}: {e}")

# Run renaming across all systems
for user_host, config in remote_configs.items():
    rename_remote_dirs(user_host, config["password"], config["subdir"])


🔄 Renaming folders in cadencea12@172.16.121.82...


✅ Renamed to: array_config_1, array_config_2
🔄 Renaming folders in cadencea1@172.16.121.72...


✅ Renamed to: array_config_5, array_config_6
🔄 Renaming folders in cadencea8@172.16.121.78...


✅ Renamed to: array_config_7, array_config_8
🔄 Renaming folders in cadencea10@172.16.121.80...


✅ Renamed to: array_config_9, array_config_10
🔄 Renaming folders in cadencea15@172.16.121.6...


✅ Renamed to: array_config_11, array_config_12
🔄 Renaming folders in cadencea14@172.16.121.84...
✅ Renamed to: array_config_13, array_config_14
🔄 Renaming folders in imt2022556_nishit@172.16.121.37...


✅ Renamed to: array_config_3, array_config_4


# Installing Numpy in all systems 

In [2]:
import subprocess
import os

# ---------------- CONFIG ---------------- #

remote_configs = {
    "cadencea12@172.16.121.82": {"password": "caduser@123", "log_prefix": "12"},
    "cadencea1@172.16.121.72": {"password": "caduser@123", "log_prefix": "1"},
    "cadencea8@172.16.121.78": {"password": "caduser@123", "log_prefix": "8"},
    "cadencea10@172.16.121.80": {"password": "caduser@123", "log_prefix": "10"},
    "cadencea15@172.16.121.6": {"password": "caduser@123", "log_prefix": "15"},
    "cadencea14@172.16.121.84": {"password": "caduser@123", "log_prefix": "14"},
    "imt2022556_nishit@172.16.121.37": {"password": "$@Rl@1234", "log_prefix": "sarl"},
}

remote_subfolders = ["array_4096_1", "array_4096_2"]
local_file = "modify_rtl.py"

# ---------------- FUNCTIONS ---------------- #

def send_file_and_install(user_host, password, subfolder):
    user, host = user_host.split("@")
    remote_path = f"/home/{user}/Documents/strassen/{subfolder}"

    # Step 1: Send modify_rtl.py
    print(f"\n📤 Sending {local_file} to {user_host}:{remote_path}/")
    scp_cmd = [
        "sshpass", "-p", password,
        "scp", "-o", "StrictHostKeyChecking=no", local_file,
        f"{user_host}:{remote_path}/"
    ]

    try:
        subprocess.run(scp_cmd, check=True)
        print("✅ File sent.")
    except subprocess.CalledProcessError:
        print("❌ File transfer failed.")
        return

    # Step 2: Install numpy
    print(f"📦 Installing numpy on {user_host}...")
    ssh_cmd = [
        "sshpass", "-p", password,
        "ssh", "-o", "StrictHostKeyChecking=no",
        f"{user_host}",
        "pip3 install --user numpy"
    ]

    try:
        subprocess.run(ssh_cmd, check=True)
        print("✅ Numpy installed.")
    except subprocess.CalledProcessError:
        print("❌ Failed to install numpy.")

# ---------------- EXECUTION ---------------- #

for user_host, config in remote_configs.items():
    for subfolder in remote_subfolders:
        send_file_and_install(user_host, config["password"], subfolder)



📤 Sending modify_rtl.py to cadencea12@172.16.121.82:/home/cadencea12/Documents/strassen/array_4096_1/
✅ File sent.
📦 Installing numpy on cadencea12@172.16.121.82...
  Using cached https://files.pythonhosted.org/packages/45/b2/6c7545bb7a38754d63048c7696804a0d947328125d81bf12beaa692c3ae3/numpy-1.19.5-cp36-cp36m-manylinux1_x86_64.whl
✅ Numpy installed.

📤 Sending modify_rtl.py to cadencea12@172.16.121.82:/home/cadencea12/Documents/strassen/array_4096_2/
✅ File sent.
📦 Installing numpy on cadencea12@172.16.121.82...
✅ Numpy installed.

📤 Sending modify_rtl.py to cadencea1@172.16.121.72:/home/cadencea1/Documents/strassen/array_4096_1/
✅ File sent.
📦 Installing numpy on cadencea1@172.16.121.72...
✅ Numpy installed.

📤 Sending modify_rtl.py to cadencea1@172.16.121.72:/home/cadencea1/Documents/strassen/array_4096_2/
✅ File sent.
📦 Installing numpy on cadencea1@172.16.121.72...
✅ Numpy installed.

📤 Sending modify_rtl.py to cadencea8@172.16.121.78:/home/cadencea8/Documents/strassen/array_4096_

# Broadcast 30MiB top.v main code

In [ ]:
import subprocess

remote_configs = {
    "cadencea12@172.16.121.82": {"password": "caduser@123", "subdir": ["array_config_1", "array_config_2"]},
    "cadencea1@172.16.121.72": {"password": "caduser@123", "subdir": ["array_config_5", "array_config_6"]},
    "cadencea8@172.16.121.78": {"password": "caduser@123", "subdir": ["array_config_7", "array_config_8"]},
    "cadencea10@172.16.121.80": {"password": "caduser@123", "subdir": ["array_config_9", "array_config_10"]},
    "cadencea15@172.16.121.6": {"password": "caduser@123", "subdir": ["array_config_11", "array_config_12"]},
    "cadencea14@172.16.121.84": {"password": "caduser@123", "subdir": ["array_config_13", "array_config_14"]},
    "imt2022556_nishit@172.16.121.37": {"password": "$@Rl@1234", "subdir": ["array_config_3", "array_config_4"]},
}

local_file = "./gen/1/rtl/top.v" # 30 MiB rtl code

for user_host, config in remote_configs.items():
    remote_user, remote_host = user_host.split("@")
    password = config["password"]
    
    for subdir in config["subdir"]:
        remote_path = f"/home/{remote_user}/Documents/strassen/{subdir}/rtl/"
        print(f"📤 Sending {local_file} → {user_host}:{remote_path}")

        scp_command = [
            "sshpass", "-p", password,
            "scp",
            "-o", "StrictHostKeyChecking=no",
            "-o", "UserKnownHostsFile=/dev/null",
            local_file,
            f"{remote_user}@{remote_host}:{remote_path}"
        ]

        result = subprocess.run(scp_command)
        if result.returncode == 0:
            print(f"✅ Overwrote top.v in {subdir}/rtl on {user_host}")
        else:
            print(f"❌ Failed to copy top.v to {user_host}:{remote_path}")


📤 Sending ./gen/1/rtl/top.v → cadencea12@172.16.121.82:/home/cadencea12/Documents/strassen/array_config_1/rtl/


✅ Overwrote top.v in array_config_1/rtl on cadencea12@172.16.121.82
📤 Sending ./gen/1/rtl/top.v → cadencea12@172.16.121.82:/home/cadencea12/Documents/strassen/array_config_2/rtl/


✅ Overwrote top.v in array_config_2/rtl on cadencea12@172.16.121.82
📤 Sending ./gen/1/rtl/top.v → cadencea1@172.16.121.72:/home/cadencea1/Documents/strassen/array_config_5/rtl/


✅ Overwrote top.v in array_config_5/rtl on cadencea1@172.16.121.72
📤 Sending ./gen/1/rtl/top.v → cadencea1@172.16.121.72:/home/cadencea1/Documents/strassen/array_config_6/rtl/


✅ Overwrote top.v in array_config_6/rtl on cadencea1@172.16.121.72
📤 Sending ./gen/1/rtl/top.v → cadencea8@172.16.121.78:/home/cadencea8/Documents/strassen/array_config_7/rtl/


...

✅ Overwrote top.v in array_config_12/rtl on cadencea15@172.16.121.6
📤 Sending ./gen/1/rtl/top.v → cadencea14@172.16.121.84:/home/cadencea14/Documents/strassen/array_config_13/rtl/


✅ Overwrote top.v in array_config_13/rtl on cadencea14@172.16.121.84
📤 Sending ./gen/1/rtl/top.v → cadencea14@172.16.121.84:/home/cadencea14/Documents/strassen/array_config_14/rtl/


✅ Overwrote top.v in array_config_14/rtl on cadencea14@172.16.121.84
📤 Sending ./gen/1/rtl/top.v → imt2022556_nishit@172.16.121.37:/home/imt2022556_nishit/Documents/strassen/array_config_3/rtl/


✅ Overwrote top.v in array_config_3/rtl on imt2022556_nishit@172.16.121.37
📤 Sending ./gen/1/rtl/top.v → imt2022556_nishit@172.16.121.37:/home/imt2022556_nishit/Documents/strassen/array_config_4/rtl/


✅ Overwrote top.v in array_config_4/rtl on imt2022556_nishit@172.16.121.37


# get_metrics.py to be broadcasted to all remotes

In [ ]:
import re
import csv

def extract_metrics(area_file, power_file, delay_file):
    # AREA
    with open(area_file, 'r') as f:
        area_text = f.read()
    area_match = re.search(r"total\s+\d+\s+([\d.]+)", area_text.strip().splitlines()[-1])
    area = float(area_match.group(1)) if area_match else None

    # POWER
    with open(power_file, 'r') as f:
        power_text = f.read()
    power_match = re.search(r"Subtotal\s+(?:\S+\s+){3}(\S+)", power_text)
    power = float(power_match.group(1)) if power_match else None

    # DELAY
    with open(delay_file, 'r') as f:
        delay_text = f.read()
    delay_match = re.search(r"^\s*C_out_reg\[\d+\]/D.*?(\d+)\s*$", delay_text, re.MULTILINE)
    delay = int(delay_match.group(1)) if delay_match else None

    return area, power, delay


area_file = './reports/SystolicArray/45nm/opt/syn_opt_gates.txt'
power_file = './reports/SystolicArray/45nm/opt/syn_opt_power.txt'
delay_file = './reports/SystolicArray/45nm/opt/syn_opt_timing.txt'

area, power, delay = extract_metrics(area_file, power_file, delay_file)
print(f"Area  : {area}")
print(f"Power : {power}")
print(f"Delay : {delay}")

with open("metrics.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["area", "power", "delay"])
    writer.writerow([area, power, delay])

# Broadcaster : Send data to all remotes

In [ ]:
import subprocess
import os

# ------------------------- CONFIGURABLE SECTION ------------------------- #
remote_configs = {
    # lagging system ( needs to push along using ipc )
    # "cadencea19@172.16.121.14": {"password": "caduser@123", "enabled": True},
    # "cadencea22@172.16.121.25": {"password": "caduser@123", "enabled": True},
    # "cadencea6@172.16.121.66": {"password": "caduser@123", "enabled": True},
    # "cadencea24@172.16.121.30": {"password": "caduser@123", "enabled": True},
    # "cadencea16@172.16.121.75": {"password": "caduser@123", "enabled": True},
    # "cadencea2@172.16.121.8": {"password": "caduser@123", "enabled": True},
    # "cadencea18@172.16.121.11": {"password": "caduser@123", "enabled": True},


    # Flawless systems
    "cadencea12@172.16.121.82": {"password": "caduser@123", "enabled": True},
    "cadencea1@172.16.121.72": {"password": "caduser@123", "enabled": True},
    "cadencea8@172.16.121.78": {"password": "caduser@123", "enabled": True},
    "imt2022556_nishit@172.16.121.37": {"password": "$@Rl@1234", "enabled": True},

    # Tested and working
    "cadencea15@172.16.121.6": {"password": "caduser@123", "enabled": True},
    "cadencea14@172.16.121.84": {"password": "caduser@123", "enabled": True},
    "cadencea10@172.16.121.80": {"password": "caduser@123", "enabled": True},
}

# ------------------------- FUNCTIONAL SECTION ------------------------- #

def broadcast(local_path, remote_path, post_commands):
    """
    Broadcast a local file/folder to multiple remote servers and run commands
    """
    if not os.path.exists(local_path):
        print(f"❌ Path '{local_path}' does not exist!")
        return

    for user_host, config in remote_configs.items():
        if not config.get("enabled", False):
            print(f"⏩ Skipping {user_host} (disabled)")
            continue

        password = config["password"]
        user, host = user_host.split("@")

        print(f"\n📡 Sending to {user_host}...")

        # Use rsync for folders, scp for files
        is_folder = os.path.isdir(local_path)
        if is_folder:
            scp_cmd = [
                "sshpass", "-p", password,
                "rsync", "-avz", "-e", "ssh -o StrictHostKeyChecking=no",
                local_path + "/", f"{user_host}:{remote_path}/"
            ]
        else:
            scp_cmd = [
                "sshpass", "-p", password,
                "scp", "-o", "StrictHostKeyChecking=no",
                local_path, f"{user_host}:{remote_path}/"
            ]

        try:
            subprocess.run(scp_cmd, check=True)
            print(f"✅ File/folder copied to {remote_path}/")

            for cmd in post_commands:
                remote_command = f"cd {remote_path} && {cmd}"
                ssh_cmd = [
                    "sshpass", "-p", password,
                    "ssh", "-o", "StrictHostKeyChecking=no",
                    user_host,
                    remote_command
                ]
                print(f"🚀 Running: {cmd}")
                subprocess.run(ssh_cmd, check=True)
            print(f"✅ Commands finished on {user_host}")

        except subprocess.CalledProcessError as e:
            print(f"❌ Failed on {user_host}: {e}")

# ------------------------- USAGE EXAMPLES ------------------------- #

if __name__ == "__main__":
    pass

    ### ======== TASK 1: Broadcast get_metrics.py to 2 folders on selected servers ========

    # local_file = "/home/nira/Downloads/get_metrics.py"
    # remote_subfolders = ["array_4096_1", "array_4096_2"]

    # for subfolder in remote_subfolders:
    #     remote_dir = f"/home/{{user}}/Documents/strassen/{subfolder}"

    #     # Dynamically replace user in path
    #     for user_host in remote_configs:
    #         if not remote_configs[user_host]["enabled"]:
    #             continue
    #         user = user_host.split("@")[0]
    #         path = remote_dir.replace("{user}", user)

    #         print(f"\n🚚 Broadcasting get_metrics.py to {user_host}:{path}")
    #         broadcast(
    #             local_path=local_file,
    #             remote_path=path,
    #             post_commands=[]
    #         )

    ### ======== TASK 2: Broadcast and unzip CIFAR zip ========

    # local_zip = "/home/nira/Downloads/cifar_python_strassen_final.zip"
    # remote_strassen_dir = f"/home/{{user}}/Documents/strassen"

    # unzip_commands = [
    #     "unzip -o cifar_python_strassen_final.zip",
    #     "rm -rf cifar_python_strassen_final.zip"
    # ]

    # for user_host in remote_configs:
    #     if not remote_configs[user_host]["enabled"]:
    #         continue
    #     user = user_host.split("@")[0]
    #     path = remote_strassen_dir.replace("{user}", user)

    #     print(f"\n🚚 Broadcasting CIFAR zip to {user_host}:{path}")
    #     broadcast(
    #         local_path=local_zip,
    #         remote_path=path,
    #         post_commands=unzip_commands
    #     )


# Broadcast : Commands to run on all remotes 

In [1]:
# Python Script: Append exit to Remote script.tcl

import subprocess

remote_configs = {
    # Flawless systems
    "cadencea12@172.16.121.82": {"password": "caduser@123", "enabled": True},
    "cadencea1@172.16.121.72": {"password": "caduser@123", "enabled": True},
    "cadencea8@172.16.121.78": {"password": "caduser@123", "enabled": True},
    "imt2022556_nishit@172.16.121.37": {"password": "$@Rl@1234", "enabled": True},

    # Tested and working
    "cadencea15@172.16.121.6": {"password": "caduser@123", "enabled": True},
    "cadencea14@172.16.121.84": {"password": "caduser@123", "enabled": True},
    "cadencea10@172.16.121.80": {"password": "caduser@123", "enabled": True},
}

remote_subfolders = ["array_4096_1", "array_4096_2"]

for user_host, config in remote_configs.items():
    if not config.get("enabled", False):
        continue

    user, host = user_host.split("@")
    password = config["password"]

    for subfolder in remote_subfolders:
        remote_file = f"/home/{user}/Documents/strassen/{subfolder}/script.tcl"

        # The command ensures `exit` is only appended if it's not already present
        remote_cmd = f"""bash -c 'grep -qxF "exit" {remote_file} || echo exit >> {remote_file}'"""

        full_command = [
            "sshpass", "-p", password,
            "ssh", "-o", "StrictHostKeyChecking=no",
            "-o", "UserKnownHostsFile=/dev/null",
            f"{user}@{host}",
            remote_cmd
        ]

        print(f"🔧 Appending 'exit' to: {user_host}:{remote_file}")
        try:
            subprocess.run(full_command, check=True)
            print(f"✅ Done: {remote_file}")
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed on {user_host}:{remote_file} → {e}")


🔧 Appending 'exit' to: cadencea12@172.16.121.82:/home/cadencea12/Documents/strassen/array_4096_1/script.tcl
✅ Done: /home/cadencea12/Documents/strassen/array_4096_1/script.tcl
🔧 Appending 'exit' to: cadencea12@172.16.121.82:/home/cadencea12/Documents/strassen/array_4096_2/script.tcl


✅ Done: /home/cadencea12/Documents/strassen/array_4096_2/script.tcl
🔧 Appending 'exit' to: cadencea1@172.16.121.72:/home/cadencea1/Documents/strassen/array_4096_1/script.tcl


✅ Done: /home/cadencea1/Documents/strassen/array_4096_1/script.tcl
🔧 Appending 'exit' to: cadencea1@172.16.121.72:/home/cadencea1/Documents/strassen/array_4096_2/script.tcl


✅ Done: /home/cadencea1/Documents/strassen/array_4096_2/script.tcl
🔧 Appending 'exit' to: cadencea8@172.16.121.78:/home/cadencea8/Documents/strassen/array_4096_1/script.tcl
✅ Done: /home/cadencea8/Documents/strassen/array_4096_1/script.tcl
🔧 Appending 'exit' to: cadencea8@172.16.121.78:/home/cadencea8/Documents/strassen/array_4096_2/script.tcl


✅ Done: /home/cadencea8/Documents/strassen/array_4096_2/script.tcl
🔧 Appending 'exit' to: imt2022556_nishit@172.16.121.37:/home/imt2022556_nishit/Documents/strassen/array_4096_1/script.tcl


...

✅ Done: /home/imt2022556_nishit/Documents/strassen/array_4096_2/script.tcl
🔧 Appending 'exit' to: cadencea15@172.16.121.6:/home/cadencea15/Documents/strassen/array_4096_1/script.tcl


✅ Done: /home/cadencea15/Documents/strassen/array_4096_1/script.tcl
🔧 Appending 'exit' to: cadencea15@172.16.121.6:/home/cadencea15/Documents/strassen/array_4096_2/script.tcl


✅ Done: /home/cadencea15/Documents/strassen/array_4096_2/script.tcl
🔧 Appending 'exit' to: cadencea14@172.16.121.84:/home/cadencea14/Documents/strassen/array_4096_1/script.tcl
✅ Done: /home/cadencea14/Documents/strassen/array_4096_1/script.tcl
🔧 Appending 'exit' to: cadencea14@172.16.121.84:/home/cadencea14/Documents/strassen/array_4096_2/script.tcl


✅ Done: /home/cadencea14/Documents/strassen/array_4096_2/script.tcl
🔧 Appending 'exit' to: cadencea10@172.16.121.80:/home/cadencea10/Documents/strassen/array_4096_1/script.tcl
✅ Done: /home/cadencea10/Documents/strassen/array_4096_1/script.tcl
🔧 Appending 'exit' to: cadencea10@172.16.121.80:/home/cadencea10/Documents/strassen/array_4096_2/script.tcl


✅ Done: /home/cadencea10/Documents/strassen/array_4096_2/script.tcl


# Broadcast a file but change one line in the file DEPENDING on the remote we are sending to

In [3]:
import subprocess
import os

# Your config
remote_configs = {
    "cadencea12@172.16.121.82": {"password": "caduser@123", "subdir": ["array_config_1", "array_config_2"]},
    "cadencea1@172.16.121.72": {"password": "caduser@123", "subdir": ["array_config_5", "array_config_6"]},
    "cadencea8@172.16.121.78": {"password": "caduser@123", "subdir": ["array_config_7", "array_config_8"]},
    "cadencea10@172.16.121.80": {"password": "caduser@123", "subdir": ["array_config_9", "array_config_10"]},
    "cadencea15@172.16.121.6": {"password": "caduser@123", "subdir": ["array_config_11", "array_config_12"]},
    "cadencea14@172.16.121.84": {"password": "caduser@123", "subdir": ["array_config_13", "array_config_14"]},
    "imt2022556_nishit@172.16.121.37": {"password": "$@Rl@1234", "subdir": ["array_config_3", "array_config_4"]},
}

# Load original template file
with open("approx_mult_modules_automate.py", "r") as f:
    original_code = f.readlines()

# Loop through each target and send customized file
for user_host, info in remote_configs.items():
    remote_user, remote_host = user_host.split("@")
    password = info["password"]

    for subdir in info["subdir"]:
        # Modify the line
        modified_code = []
        for line in original_code:
            if line.strip().startswith("arr = np.loadtxt("):
                modified_code.append(f"arr = np.loadtxt('{subdir}.dat', dtype=np.int8)\n")
            else:
                modified_code.append(line)

        # Save the modified file temporarily
        temp_filename = f"temp_{subdir}.py"
        with open(temp_filename, "w") as f:
            f.writelines(modified_code)

        # Construct remote path
        remote_path = f"/home/{remote_user}/Documents/strassen/{subdir}/"

        print(f"📤 Sending {temp_filename} → {remote_user}@{remote_host}:{remote_path}")

        scp_cmd = [
            "sshpass", "-p", password,
            "scp",
            "-o", "StrictHostKeyChecking=no",
            "-o", "UserKnownHostsFile=/dev/null",
            temp_filename,
            f"{remote_user}@{remote_host}:{remote_path}approx_mult_modules_automate.py"
        ]

        result = subprocess.run(scp_cmd)

        if result.returncode == 0:
            print(f"✅ Sent customized script for {subdir}")
        else:
            print(f"❌ Failed to send script for {subdir}")

        os.remove(temp_filename)  # Clean up


📤 Sending temp_array_config_1.py → cadencea12@172.16.121.82:/home/cadencea12/Documents/strassen/array_config_1/


✅ Sent customized script for array_config_1
📤 Sending temp_array_config_2.py → cadencea12@172.16.121.82:/home/cadencea12/Documents/strassen/array_config_2/
✅ Sent customized script for array_config_2
📤 Sending temp_array_config_5.py → cadencea1@172.16.121.72:/home/cadencea1/Documents/strassen/array_config_5/


✅ Sent customized script for array_config_5
📤 Sending temp_array_config_6.py → cadencea1@172.16.121.72:/home/cadencea1/Documents/strassen/array_config_6/
✅ Sent customized script for array_config_6
📤 Sending temp_array_config_7.py → cadencea8@172.16.121.78:/home/cadencea8/Documents/strassen/array_config_7/


✅ Sent customized script for array_config_7
📤 Sending temp_array_config_8.py → cadencea8@172.16.121.78:/home/cadencea8/Documents/strassen/array_config_8/
✅ Sent customized script for array_config_8
📤 Sending temp_array_config_9.py → cadencea10@172.16.121.80:/home/cadencea10/Documents/strassen/array_config_9/


✅ Sent customized script for array_config_9
📤 Sending temp_array_config_10.py → cadencea10@172.16.121.80:/home/cadencea10/Documents/strassen/array_config_10/
✅ Sent customized script for array_config_10
📤 Sending temp_array_config_11.py → cadencea15@172.16.121.6:/home/cadencea15/Documents/strassen/array_config_11/


✅ Sent customized script for array_config_11
📤 Sending temp_array_config_12.py → cadencea15@172.16.121.6:/home/cadencea15/Documents/strassen/array_config_12/
✅ Sent customized script for array_config_12
📤 Sending temp_array_config_13.py → cadencea14@172.16.121.84:/home/cadencea14/Documents/strassen/array_config_13/


✅ Sent customized script for array_config_13
📤 Sending temp_array_config_14.py → cadencea14@172.16.121.84:/home/cadencea14/Documents/strassen/array_config_14/
✅ Sent customized script for array_config_14
📤 Sending temp_array_config_3.py → imt2022556_nishit@172.16.121.37:/home/imt2022556_nishit/Documents/strassen/array_config_3/


✅ Sent customized script for array_config_3
📤 Sending temp_array_config_4.py → imt2022556_nishit@172.16.121.37:/home/imt2022556_nishit/Documents/strassen/array_config_4/
✅ Sent customized script for array_config_4
